## Intro


In this notebook, we'll take a look at NBA salaries since 1990-91 all the way up to today, we'll check how did the share of contracts change through the seasons

## Retrieving salaries

In [10]:
import requests
from bs4 import BeautifulSoup
import time
import tqdm
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
resp = requests.get("https://hoopshype.com/salaries/players/2005-2006/")

In [3]:
soup = BeautifulSoup(resp.text)

In [4]:
table_div = soup.find_all(name='div', attrs={'class': 'hh-salaries-ranking'})[0]

In [5]:
table = list(table_div.children)[1]

In [6]:
table_body = list(table.children)[3]

In [7]:
rows = table_body.find_all('tr')

In [9]:
for item in rows:
    print(list(item.find_all('td')))
    print("======")
    print(list(item.find_all('td'))[1].text.strip())
    print(list(item.find_all('td'))[2].text.strip()[1:].replace(",", ""))
    break

[<td class="rank">
						1.
					</td>, <td class="name">
<a href="https://hoopshype.com/player/shaquille-oneal/salary/">
								Shaquille O'Neal							</a>
</td>, <td class="" data-value="20000000" style="color:black">
							$20,000,000						</td>, <td class="" data-value="26508688" style="color:black">
							$26,508,688						</td>]
Shaquille O'Neal
20000000


In [10]:
def get_players_salaries(season):
    resp = requests.get(f"https://hoopshype.com/salaries/players/{season}")

    soup = BeautifulSoup(resp.text)
    table_div = soup.find_all(name='div', attrs={'class': 'hh-salaries-ranking'})[0]
    table = list(table_div.children)[1]
    table_body = list(table.children)[3]
    rows = table_body.find_all('tr')
    salaries_items = []
    for item in rows:
        salaries_items.append(
            {
                'SEASON': '2020-2021' if season == '' else season,
                'PLAYER': list(item.find_all('td'))[1].text.strip(),
                'SALARY': int(list(item.find_all('td'))[2].text.strip()[1:].replace(",", ""))
            }
        )
    return salaries_items

In [42]:
seasons = [f"{item}-{item+1}" for item in range(1990, 2021)]

In [49]:
total_items = []
for season in tqdm.tqdm(seasons):
    current_items = get_players_salaries(season)
    total_items = total_items + current_items
    time.sleep(0.5)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 31/31 [01:10<00:00,  2.26s/it]


In [52]:
data_frame = pd.DataFrame(total_items)

In [54]:
data_frame.to_csv('salaries.csv')

### Loading Salaries

In [31]:
data_frame = pd.read_csv('salaries.csv')

In [29]:
#data_frame = data_frame.loc[:, ['SEASON', 'PLAYER', 'SALARY']]

In [30]:
#data_frame.loc[:, 'SEASON'] = [f"{item.split('-')[0]}-{item.split('-')[1][-2:]}" for item in data_frame.SEASON]

In [28]:
data_frame.to_csv('salaries.csv', index=False)

In [32]:
data_frame

,SEASON,PLAYER,SALARY
0,1990-91,Patrick Ewing,4250000
1,1990-91,Hot Rod Williams,3785000
2,1990-91,Hakeem Olajuwon,3175000
3,1990-91,Charles Barkley,2900000
4,1990-91,Chris Mullin,2850000
5,1990-91,Isiah Thomas,2720000
6,1990-91,Danny Ferry,2640000
7,1990-91,Robert Parish,2500000
8,1990-91,Michael Jordan,2500000
9,1990-91,Moses Malone,2406000


#### Salary cap download

In [2]:
text = """
<div class="sr_share_wrap" style="overflow:auto">
<table class="sr_share" id="" data-cols-to-freeze=",1" style="border-collapse: collapse; font-size:.83em; border: 1px sold #aaa; overflow:auto;"><thead></thead>
    <caption>Salary Cap History Table</caption>
      <tbody>
      <tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_1985.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">1984-85</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$3,600,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$9,083,539</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_1986.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">1985-86</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$4,233,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$10,479,694</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_1987.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">1986-87</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$4,945,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$11,812,089</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_1988.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">1987-88</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$6,164,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$14,143,899</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_1989.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">1988-89</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$7,232,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$15,832,740</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_1990.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">1989-90</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$9,802,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$20,360,496</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_1991.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">1990-91</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$11,871,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$23,652,090</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_1992.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">1991-92</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$12,500,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$24,172,990</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_1993.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">1992-93</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$14,000,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$26,300,391</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_1994.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">1993-94</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$15,175,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$27,784,491</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_1995.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">1994-95</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$15,964,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$28,431,190</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_1996.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">1995-96</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$23,000,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$39,797,368</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_1997.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">1996-97</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$24,363,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$41,185,792</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_1998.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">1997-98</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$26,900,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$44,787,566</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_1999.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">1998-99</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$30,000,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$48,871,354</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_2000.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">1999-00</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$34,000,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$53,578,743</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_2001.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">2000-01</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$35,500,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$54,407,536</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_2002.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">2001-02</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$42,500,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$64,123,496</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_2003.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">2002-03</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$40,271,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$59,408,466</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_2004.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">2003-04</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$43,840,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$62,987,174</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_2005.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">2004-05</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$43,870,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$60,957,548</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_2006.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">2005-06</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$49,500,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$66,634,615</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_2007.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">2006-07</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$53,135,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$69,541,955</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_2008.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">2007-08</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$55,630,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$70,117,874</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_2009.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">2008-09</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$58,680,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$74,226,058</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_2010.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">2009-10</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$57,700,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$71,805,814</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_2011.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">2010-11</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$58,044,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$70,022,014</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_2012.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">2011-12</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$58,044,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$68,611,221</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_2013.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">2012-13</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$58,044,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$67,621,260</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_2014.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">2013-14</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$58,679,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$67,265,567</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_2015.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">2014-15</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$63,065,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$72,202,032</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_2016.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">2015-16</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$70,000,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$79,141,827</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_2017.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">2016-17</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$94,143,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$104,227,011</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_2018.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">2017-18</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$99,093,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$107,090,301</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_2019.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">2018-19</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$101,869,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$108,127,254</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_2020.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">2019-20</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$109,140,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$114,444,734</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_2021.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">2020-21</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$109,140,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$109,140,000</td></tr>
<tr><th scope="row" style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "><a href="https://www.basketball-reference.com/leagues/NBA_2022.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">2021-22</a></th><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; ">$112,414,000</td><td style="border: 1px solid #ccc; padding: 2px 3px 2px 2px; white-space: nowrap; "></td></tr>
</tbody></table><div id="credit_salary_cap_history" class="sr_share" style="font-size: 0.83em;">Provided by <a href="https://www.sports-reference.com/sharing.html?utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool">Basketball-Reference.com</a>: <a href="https://www.basketball-reference.com/contracts/salary-cap-history.html?sr&amp;utm_source=direct&amp;utm_medium=Share&amp;utm_campaign=ShareTool#salary_cap_history">View Original Table</a><br>Generated 10/7/2021.</div>

"""

In [3]:
soup = BeautifulSoup(text)

In [4]:
values = {
    'SEASON': [],
    'CAP': []
}
for row in soup.find("table").findAllNext('tr'):
    data = list(row.children)
    
    values['SEASON'].append(data[0].text)
    values['CAP'].append(int(data[1].text[1:].replace(",", "")))

In [5]:
cap_df = pd.DataFrame(values)

In [6]:
cap_df

,SEASON,CAP
0,1984-85,3600000
1,1985-86,4233000
2,1986-87,4945000
3,1987-88,6164000
4,1988-89,7232000
5,1989-90,9802000
6,1990-91,11871000
7,1991-92,12500000
8,1992-93,14000000
9,1993-94,15175000


#### Full data

In [3]:
full_df = pd.merge(left=data_frame, right=cap_df, on=['SEASON'])

NameError: name 'data_frame' is not defined

In [4]:

full_df.to_csv('salaries_cap.csv', index=False)

NameError: name 'full_df' is not defined

## Loading full data

In [5]:
full_df = pd.read_csv('salaries_cap.csv')

In [7]:
full_df.loc[:, 'CAP_PCT'] = full_df.SALARY / full_df.CAP * 100

In [11]:
full_df.loc[(full_df.SEASON == '1994-95')]

,SEASON,PLAYER,SALARY,CAP,CAP_PCT
1521,1994-95,Magic Johnson,14660000,15964000,91.831621
1522,1994-95,David Robinson,7300000,15964000,45.727888
1523,1994-95,James Worthy,7250000,15964000,45.414683
1524,1994-95,A.C. Green,6472600,15964000,40.544976
1525,1994-95,Detlef Schrempf,5000000,15964000,31.320471
1526,1994-95,Shaquille O'Neal,4800000,15964000,30.067652
1527,1994-95,Patrick Ewing,4486700,15964000,28.105112
1528,1994-95,Anfernee Hardaway,4305000,15964000,26.966926
1529,1994-95,Derrick Coleman,4212000,15964000,26.384365
1530,1994-95,John Williams,4161000,15964000,26.064896


## Data from bbref

#### Getting team links

In [11]:
text = requests.get('https://www.basketball-reference.com/teams/')

In [12]:
soup = BeautifulSoup(text.text)

In [13]:
table = soup.findAll('table')[0]

In [14]:
team_dict = {}

In [15]:
for row in table.findAll('tr'):
    heading = row.findAll('th')[0]
    link = heading.a
    if link is None:
        continue
    team_dict[link.attrs['href'].rsplit('/')[-2]] = link.text

## Getting salaries data

In [18]:
seasons = list(range(1991, 2023))

In [23]:
seasons

[1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021,
 2022]

In [92]:
resp = requests.get('https://www.basketball-reference.com/teams/DEN/1991.html')

In [93]:
text = resp.text

In [117]:
text = text.replace('!--', '').replace('-->', '')

In [118]:
soup = BeautifulSoup(text)

In [128]:
table = soup.findAll('div', attrs={'id': 'all_salaries2'})[0].find('div', attrs={'id': 'div_salaries2'}).table

In [9]:
alts = {
    'NJN': ['BRK'],
    'OKC': ['SEA'],
    'WAS': ['WSB'],
    'CHA': ['CHH', 'CHO'],
    'NOH': ['NOK', 'NOP'],
    'MEM': ['VAN'],
    'TOR': []
}

In [16]:
team_dict.keys()

dict_keys(['ATL', 'BOS', 'NJN', 'CHA', 'CHI', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOH', 'NYK', 'OKC', 'ORL', 'PHI', 'PHO', 'POR', 'SAC', 'SAS', 'TOR', 'UTA', 'WAS'])

In [20]:
def get_soup(url):
    resp = requests.get(url)
    text = resp.text
    text = text.replace('!--', '').replace('-->', '')
    soup = BeautifulSoup(text)
    return soup

data = {
    'SEASON': [],
    'TEAM': [],
    'PLAYER': [],
    'SALARY': []
}
for season in seasons:
    print(season)
    for team_abbr in team_dict:
        url = f"https://www.basketball-reference.com/teams/{team_abbr}/{season}.html"
        soup = get_soup(url)
        tables = soup.findAll('div', attrs={'id': 'all_salaries2'})
        print(team_abbr)
        if len(tables) == 0:
            found = False
            for alt in alts[team_abbr]:
                print(alt)
                soup = get_soup(f"https://www.basketball-reference.com/teams/{alt}/{season}.html")
                tables = soup.findAll('div', attrs={'id': 'all_salaries2'})
                if len(tables) > 0:
                    found = True
                    team_abbr = alt
                    break
            if not found:
                print(f"Not found: {team_abbr}-{season}")
                continue
        table = tables[0].find('div', attrs={'id': 'div_salaries2'}).table
        for row in table.findAll('tr'):
            #print(row)
            cells = row.findAll('td')
            if len(cells) == 0:
                continue
            salary = cells[1].text[1:].replace(',', '')
            if len(salary) == 0:
                continue
            data['SEASON'].append(f"{season}-{str(int(season)+1)[-2:]}")
            data['TEAM'].append(team_abbr)
            data['PLAYER'].append(cells[0].text)
            data['SALARY'].append(int(salary))
        time.sleep(0.1)

1991
ATL
BOS
NJN
CHA
CHH
CHI
CLE
DAL
DEN
DET
GSW
HOU
IND
LAC
LAL
MEM
VAN
Not found: MEM-1991
MIA
MIL
MIN
NOH
NOK
NOP
Not found: NOH-1991
NYK
OKC
SEA
ORL
PHI
PHO
POR
SAC
SAS
TOR
Not found: TOR-1991
UTA
WAS
WSB
1992
ATL
BOS
NJN
CHA
CHH
CHI
CLE
DAL
DEN
DET
GSW
HOU
IND
LAC
LAL
MEM
VAN
Not found: MEM-1992
MIA
MIL
MIN
NOH
NOK
NOP
Not found: NOH-1992
NYK
OKC
SEA
ORL
PHI
PHO
POR
SAC
SAS
TOR
Not found: TOR-1992
UTA
WAS
WSB
1993
ATL
BOS
NJN
CHA
CHH
CHI
CLE
DAL
DEN
DET
GSW
HOU
IND
LAC
LAL
MEM
VAN
Not found: MEM-1993
MIA
MIL
MIN
NOH
NOK
NOP
Not found: NOH-1993
NYK
OKC
SEA
ORL
PHI
PHO
POR
SAC
SAS
TOR
Not found: TOR-1993
UTA
WAS
WSB
1994
ATL
BOS
NJN
CHA
CHH
CHI
CLE
DAL
DEN
DET
GSW
HOU
IND
LAC
LAL
MEM
VAN
Not found: MEM-1994
MIA
MIL
MIN
NOH
NOK
NOP
Not found: NOH-1994
NYK
OKC
SEA
ORL
PHI
PHO
POR
SAC
SAS
TOR
Not found: TOR-1994
UTA
WAS
WSB
1995
ATL
BOS
NJN
CHA
CHH
CHI
CLE
DAL
DEN
DET
GSW
HOU
IND
LAC
LAL
MEM
VAN
Not found: MEM-1995
MIA
MIL
MIN
NOH
NOK
NOP
Not found: NOH-1995
NYK
OKC
SEA
ORL
PHI
PHO
POR

In [161]:
data['PLAYER'][-5:]

['Aaron Brooks',
 "E'Twaun Moore",
 'Cameron Bairstow',
 'Richard Hamilton',
 'Tornike Shengelia']

In [21]:
df = pd.DataFrame(data)

In [26]:
modified_seasons = []
for item in df.SEASON.tolist():
    first_year = int(item.split('-')[0]) - 1
    new_season = f"{first_year}-{str(first_year+1)[-2:]}"
    modified_seasons.append(new_season)

In [28]:
df.loc[:, 'SEASON'] = modified_seasons

In [30]:
merged_df = pd.merge(left=df, right=cap_df, on='SEASON')

In [32]:
merged_df.to_csv('salary_data_full.csv', index=False)

## Loading full df

In [3]:
full_df = pd.read_csv('salary_data_full.csv')

In [4]:
full_df

,SEASON,TEAM,PLAYER,SALARY,CAP
0,1990-91,ATL,Moses Malone,2406000,11871000
1,1990-91,ATL,Dominique Wilkins,2065000,11871000
2,1990-91,ATL,Jon Koncak,1550000,11871000
3,1990-91,ATL,Doc Rivers,895000,11871000
4,1990-91,ATL,Rumeal Robinson,800000,11871000
5,1990-91,ATL,Tim McCormick,775000,11871000
6,1990-91,ATL,Kevin Willis,685000,11871000
7,1990-91,ATL,Alexander Volkov,650000,11871000
8,1990-91,ATL,John Battle,590000,11871000
9,1990-91,ATL,Sidney Moncrief,510000,11871000


In [25]:
full_df.loc[:, 'PCT_CAP'] = full_df.SALARY / full_df.CAP * 100

In [20]:
full_df.sort_values(by='PCT_CAP', ascending=False)

,SEASON,TEAM,PLAYER,SALARY,CAP,PCT_CAP
2412,1996-97,CHI,Michael Jordan,30140000,24363000,123.712187
2828,1997-98,CHI,Michael Jordan,33140000,26900000,123.197026
2208,1995-96,NYK,Patrick Ewing,18724000,23000000,81.408696
3049,1997-98,NYK,Patrick Ewing,20500000,26900000,76.208178
5840,2003-04,MIN,Kevin Garnett,28000000,43840000,63.868613
6289,2004-05,MIA,Shaquille O'Neal,27696430,43870000,63.132961
5378,2002-03,MIN,Kevin Garnett,25200000,40271000,62.576047
3489,1998-99,NYK,Patrick Ewing,18500000,30000000,61.666667
2639,1996-97,ORL,Horace Grant,14857000,24363000,60.981817
5318,2002-03,LAL,Shaquille O'Neal,23571429,40271000,58.532018


## Plan

#### Team level

* Iterate over each team
* Take top 12 in salaries (we can assume they play?)
* Calculate mean, stddev, top 3, bottom 3, whatever you can think of
* Visualize all that

#### League level
* Top payed players in 


In [26]:
cur_season = full_df.loc[full_df.SEASON == '2021-22']

In [27]:
cur_season.TEAM.unique()

array(['ATL', 'BOS', 'BRK', 'CHO', 'CHI', 'CLE', 'DAL', 'DEN', 'DET',
       'GSW', 'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN',
       'NOP', 'NYK', 'OKC', 'ORL', 'PHI', 'PHO', 'POR', 'SAC', 'SAS',
       'TOR', 'UTA', 'WAS'], dtype=object)

In [35]:
full_df.loc[(full_df.SEASON == '2006-07') & (full_df.TEAM == 'CHI')]

,SEASON,TEAM,PLAYER,SALARY,CAP,PCT_CAP
7097,2006-07,CHI,Ben Wallace,16000000,53135000,30.111979
7098,2006-07,CHI,P.J. Brown,8560000,53135000,16.109909
7099,2006-07,CHI,Andres Nocioni,3950000,53135000,7.433895
7100,2006-07,CHI,Ben Gordon,3862080,53135000,7.268429
7101,2006-07,CHI,Tyrus Thomas,3260760,53135000,6.136746
7102,2006-07,CHI,Kirk Hinrich,3192628,53135000,6.008522
7103,2006-07,CHI,Chris Duhon,3024000,53135000,5.691164
7104,2006-07,CHI,Mike Sweetney,2696956,53135000,5.075668
7105,2006-07,CHI,Luol Deng,2614440,53135000,4.920373
7106,2006-07,CHI,Malik Allen,1803600,53135000,3.394373


In [30]:
cur_season.loc[cur_season.TEAM == 'UTA'].sort_values(by='SALARY', ascending=False)

,SEASON,TEAM,PLAYER,SALARY,CAP,PCT_CAP
15625,2021-22,UTA,Rudy Gobert,35344828,112414000,31.441660
15626,2021-22,UTA,Donovan Mitchell,28103550,112414000,25.000044
15627,2021-22,UTA,Mike Conley,21000000,112414000,18.680947
15628,2021-22,UTA,Bojan Bogdanović,18700000,112414000,16.634939
15629,2021-22,UTA,Joe Ingles,14000000,112414000,12.453965
15630,2021-22,UTA,Jordan Clarkson,12420000,112414000,11.048446
15631,2021-22,UTA,Royce O'Neale,8678571,112414000,7.720187
15632,2021-22,UTA,Rudy Gay,5890000,112414000,5.239561
15633,2021-22,UTA,Hassan Whiteside,2401537,112414000,2.136333
15634,2021-22,UTA,Udoka Azubuike,2075880,112414000,1.846638


## Maybe idea will fall to me